### Get titles from URLS

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Load your CSV file into the DataFrame
df = pd.read_csv('labels.csv')
df

,GLOBALEVENTID,SQLDATE,MonthYear,Year,Actor1Name,Actor1CountryCode,Actor2Name,Actor2CountryCode,EventCode,NumMentions,NumSources,NumArticles,AvgTone,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,SOURCEURL,Municipality code
0,467522672,20150916,201509,2015,COLOMBIA,COL,RADIO STATION,NaN,193,4,1,4,-3.030303,CO,CO19,4.23161,-73.5079,10864086,http://www.unmultimedia.org/radio/english/2015...,Villavicencio
1,465775735,20150911,201509,2015,GUNMAN,NaN,COLOMBIA,COL,193,6,2,6,-5.497685,CO,CO19,4.23161,-73.5079,10864086,http://www.nzherald.co.nz/world/news/article.c...,Villavicencio
2,459806479,20150821,201508,2015,COLOMBIAN,COL,NaN,NaN,186,14,6,14,-2.199586,CO,CO19,4.00000,-73.3667,-589367,http://globenewswire.com/news-release/2015/08/...,Villavicencio
3,520388626,20160312,201603,2016,COLOMBIA,COL,JOURNALIST,NaN,193,2,1,2,-8.240535,CO,CO19,4.23161,-73.5079,10864086,http://www.dailystar.co.uk/news/latest-news/50...,Villavicencio
4,496403031,20151222,201512,2015,GOVERNMENT,NaN,COLOMBIA,COL,190,1,1,1,-5.557300,CO,CO19,4.32900,-72.8381,-590973,http://www.offnews.info/verArticulo.php?conten...,Villavicencio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31056,954996616,20201118,202011,2020,HONDURAS,HND,NaN,NaN,192,8,1,8,-5.488621,CO,CO21,8.80389,-72.9536,29198,https://www.marshallindependent.com/news/inter...,Cúcuta
31057,955046947,20201118,202011,2020,PRESIDENT,NaN,NaN,NaN,190,3,1,3,-4.580153,CO,CO21,8.80389,-72.9536,29198,https://colombiareports.com/colombia-rescues-d...,Cúcuta
31058,956040249,20201124,202011,2020,TOURIST,NaN,NaN,NaN,193,2,1,2,-2.453271,CO,CO21,8.80389,-72.9536,29198,https://www.laprensalatina.com/iota-deals-anot...,Cúcuta
31059,962369859,20210102,202101,2021,CARRIER,NaN,NaN,NaN,190,5,1,5,-1.003344,CO,CO21,8.27556,-72.3958,230744,https://www.flightglobal.com/safety/avianca-a3...,Cúcuta


### Filter to dates between 2018 and 2023

In [3]:
# filter sql date to get values between 2018 and 2022
df = df[(df['SQLDATE'] >= 20180000) & (df['SQLDATE'] <= 20230000)]
df

,GLOBALEVENTID,SQLDATE,MonthYear,Year,Actor1Name,Actor1CountryCode,Actor2Name,Actor2CountryCode,EventCode,NumMentions,NumSources,NumArticles,AvgTone,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,SOURCEURL,Municipality code
65,784113235,20180903,201809,2018,FARC,COL,NaN,NaN,190,5,1,5,-9.230769,CO,CO19,2.88915,-72.1256,6197118,https://morningstaronline.co.uk/article/colomb...,Villavicencio
66,784113310,20180903,201809,2018,CIVILIAN,NaN,NaN,NaN,190,3,1,3,-9.230769,CO,CO19,2.88915,-72.1256,6197118,https://morningstaronline.co.uk/article/colomb...,Villavicencio
67,784112833,20180903,201809,2018,NaN,NaN,FARC,COL,190,5,1,5,-9.230769,CO,CO19,2.88915,-72.1256,6197118,https://morningstaronline.co.uk/article/colomb...,Villavicencio
100,902558755,20200130,202001,2020,POLICE,NaN,NaN,NaN,191,5,1,5,-4.023845,CO,CO19,4.23161,-73.5079,10864086,https://www.readingeagle.com/living/english-po...,Villavicencio
107,1060909000,20220831,202208,2022,COLOMBIA,COL,NaN,NaN,193,2,1,2,-5.471125,CO,CO19,4.00000,-73.3667,-589367,https://ifex.org/united-for-salman-rushdie-and...,Villavicencio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31056,954996616,20201118,202011,2020,HONDURAS,HND,NaN,NaN,192,8,1,8,-5.488621,CO,CO21,8.80389,-72.9536,29198,https://www.marshallindependent.com/news/inter...,Cúcuta
31057,955046947,20201118,202011,2020,PRESIDENT,NaN,NaN,NaN,190,3,1,3,-4.580153,CO,CO21,8.80389,-72.9536,29198,https://colombiareports.com/colombia-rescues-d...,Cúcuta
31058,956040249,20201124,202011,2020,TOURIST,NaN,NaN,NaN,193,2,1,2,-2.453271,CO,CO21,8.80389,-72.9536,29198,https://www.laprensalatina.com/iota-deals-anot...,Cúcuta
31059,962369859,20210102,202101,2021,CARRIER,NaN,NaN,NaN,190,5,1,5,-1.003344,CO,CO21,8.27556,-72.3958,230744,https://www.flightglobal.com/safety/avianca-a3...,Cúcuta


In [4]:
# Set a global variable to keep track of the number of articles we've fetched
COUNTER = 0

def get_article_text_and_title(url):
    # access global variable
    global COUNTER
    
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extracting text
        article_text = ' '.join([p.text for p in soup.find_all('p')])

        # Attempt to extract a title from common tags
        title_tag = soup.find(['h1', 'h2', 'h3']) or soup.title
        page_title = title_tag.text.strip() if title_tag else 'No Title Found'
        
        if COUNTER % 100 == 0:
            print(f"Fetching article {COUNTER}")

        COUNTER += 1

        return article_text, page_title
    
    except requests.Timeout:
        print(f"Timeout while fetching {url}")
        return None, None
    
    except requests.RequestException as e:
        #print(f"Error fetching {url}: {e}")
        
        if COUNTER % 100 == 0:
            print(f"Fetching article {COUNTER}")
        COUNTER += 1
        return None, None
    

In [ ]:
# Apply the function and split the results into two columns
df[['ExtractedText', 'ContentTitle']] = df['SOURCEURL'].apply(
    lambda url: pd.Series(get_article_text_and_title(url))
)

In [6]:
df

,GLOBALEVENTID,SQLDATE,MonthYear,Year,Actor1Name,Actor1CountryCode,Actor2Name,Actor2CountryCode,EventCode,NumMentions,...,AvgTone,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,SOURCEURL,Municipality code,ExtractedText,ContentTitle
65,784113235,20180903,201809,2018,FARC,COL,NaN,NaN,190,5,...,-9.230769,CO,CO19,2.88915,-72.1256,6197118,https://morningstaronline.co.uk/article/colomb...,Villavicencio,Reset on: COLOMBIA brought charges against b...,LATEST EDITORIAL
66,784113310,20180903,201809,2018,CIVILIAN,NaN,NaN,NaN,190,3,...,-9.230769,CO,CO19,2.88915,-72.1256,6197118,https://morningstaronline.co.uk/article/colomb...,Villavicencio,Reset on: COLOMBIA brought charges against b...,LATEST EDITORIAL
67,784112833,20180903,201809,2018,NaN,NaN,FARC,COL,190,5,...,-9.230769,CO,CO19,2.88915,-72.1256,6197118,https://morningstaronline.co.uk/article/colomb...,Villavicencio,Reset on: COLOMBIA brought charges against b...,LATEST EDITORIAL
100,902558755,20200130,202001,2020,POLICE,NaN,NaN,NaN,191,5,...,-4.023845,CO,CO19,4.23161,-73.5079,10864086,https://www.readingeagle.com/living/english-po...,Villavicencio,None,None
107,1060909000,20220831,202208,2022,COLOMBIA,COL,NaN,NaN,193,2,...,-5.471125,CO,CO19,4.00000,-73.3667,-589367,https://ifex.org/united-for-salman-rushdie-and...,Villavicencio,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31056,954996616,20201118,202011,2020,HONDURAS,HND,NaN,NaN,192,8,...,-5.488621,CO,CO21,8.80389,-72.9536,29198,https://www.marshallindependent.com/news/inter...,Cúcuta,"Nov 18, 2020 TEGUCIGALPA, Honduras (AP) — Hurr...",Hurricane Iota roars onto Nicaragua as 2nd blo...
31057,955046947,20201118,202011,2020,PRESIDENT,NaN,NaN,NaN,190,3,...,-4.580153,CO,CO21,8.80389,-72.9536,29198,https://colombiareports.com/colombia-rescues-d...,Cúcuta,Colombia’s air force rescued more than 50 peop...,Colombia rescues dozens from island destroyed ...
31058,956040249,20201124,202011,2020,TOURIST,NaN,NaN,NaN,193,2,...,-2.453271,CO,CO21,8.80389,-72.9536,29198,https://www.laprensalatina.com/iota-deals-anot...,Cúcuta,"By Klarem Valoyes Gutierrez San Andres, Colomb...",Iota deals another blow to Colombian archipela...
31059,962369859,20210102,202101,2021,CARRIER,NaN,NaN,NaN,190,5,...,-1.003344,CO,CO21,8.27556,-72.3958,230744,https://www.flightglobal.com/safety/avianca-a3...,Cúcuta,None,None


In [7]:
# Save the DataFrame to a new CSV file
df.to_csv('labels_full.csv', index=False)

### Domestic Violence Cohort

In [5]:
df = pd.read_csv('labels_full.csv')
df

,GLOBALEVENTID,SQLDATE,MonthYear,Year,Actor1Name,Actor1CountryCode,Actor2Name,Actor2CountryCode,EventCode,NumMentions,...,AvgTone,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,SOURCEURL,Municipality code,ExtractedText,ContentTitle
0,784113235,20180903,201809,2018,FARC,COL,NaN,NaN,190,5,...,-9.230769,CO,CO19,2.88915,-72.1256,6197118,https://morningstaronline.co.uk/article/colomb...,Villavicencio,Reset on: COLOMBIA brought charges against b...,LATEST EDITORIAL
1,784113310,20180903,201809,2018,CIVILIAN,NaN,NaN,NaN,190,3,...,-9.230769,CO,CO19,2.88915,-72.1256,6197118,https://morningstaronline.co.uk/article/colomb...,Villavicencio,Reset on: COLOMBIA brought charges against b...,LATEST EDITORIAL
2,784112833,20180903,201809,2018,NaN,NaN,FARC,COL,190,5,...,-9.230769,CO,CO19,2.88915,-72.1256,6197118,https://morningstaronline.co.uk/article/colomb...,Villavicencio,Reset on: COLOMBIA brought charges against b...,LATEST EDITORIAL
3,902558755,20200130,202001,2020,POLICE,NaN,NaN,NaN,191,5,...,-4.023845,CO,CO19,4.23161,-73.5079,10864086,https://www.readingeagle.com/living/english-po...,Villavicencio,NaN,NaN
4,1060909000,20220831,202208,2022,COLOMBIA,COL,NaN,NaN,193,2,...,-5.471125,CO,CO19,4.00000,-73.3667,-589367,https://ifex.org/united-for-salman-rushdie-and...,Villavicencio,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16020,954996616,20201118,202011,2020,HONDURAS,HND,NaN,NaN,192,8,...,-5.488621,CO,CO21,8.80389,-72.9536,29198,https://www.marshallindependent.com/news/inter...,Cúcuta,"Nov 18, 2020 TEGUCIGALPA, Honduras (AP) — Hurr...",Hurricane Iota roars onto Nicaragua as 2nd blo...
16021,955046947,20201118,202011,2020,PRESIDENT,NaN,NaN,NaN,190,3,...,-4.580153,CO,CO21,8.80389,-72.9536,29198,https://colombiareports.com/colombia-rescues-d...,Cúcuta,Colombia’s air force rescued more than 50 peop...,Colombia rescues dozens from island destroyed ...
16022,956040249,20201124,202011,2020,TOURIST,NaN,NaN,NaN,193,2,...,-2.453271,CO,CO21,8.80389,-72.9536,29198,https://www.laprensalatina.com/iota-deals-anot...,Cúcuta,"By Klarem Valoyes Gutierrez San Andres, Colomb...",Iota deals another blow to Colombian archipela...
16023,962369859,20210102,202101,2021,CARRIER,NaN,NaN,NaN,190,5,...,-1.003344,CO,CO21,8.27556,-72.3958,230744,https://www.flightglobal.com/safety/avianca-a3...,Cúcuta,NaN,NaN


In [7]:
# Define EventCodes related to domestic or gender violence
# 182: Engage in domestic protest
# 184: Demonstrate for women's rights
# 185: Demonstrate for gender equality
# 186: Engage in violent protest
# 187: Engage in political dissent, not specified below (can include gender-related issues)
# 190: Advocate for social or political change
related_event_codes = [182, 184, 185, 186, 187, 190]

# Filter the DataFrame to include events with related EventCodes
filtered_df = df[df['EventCode'].isin(related_event_codes)]
filtered_df

,GLOBALEVENTID,SQLDATE,MonthYear,Year,Actor1Name,Actor1CountryCode,Actor2Name,Actor2CountryCode,EventCode,NumMentions,...,AvgTone,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,SOURCEURL,Municipality code,ExtractedText,ContentTitle
0,784113235,20180903,201809,2018,FARC,COL,NaN,NaN,190,5,...,-9.230769,CO,CO19,2.88915,-72.1256,6197118,https://morningstaronline.co.uk/article/colomb...,Villavicencio,Reset on: COLOMBIA brought charges against b...,LATEST EDITORIAL
1,784113310,20180903,201809,2018,CIVILIAN,NaN,NaN,NaN,190,3,...,-9.230769,CO,CO19,2.88915,-72.1256,6197118,https://morningstaronline.co.uk/article/colomb...,Villavicencio,Reset on: COLOMBIA brought charges against b...,LATEST EDITORIAL
2,784112833,20180903,201809,2018,NaN,NaN,FARC,COL,190,5,...,-9.230769,CO,CO19,2.88915,-72.1256,6197118,https://morningstaronline.co.uk/article/colomb...,Villavicencio,Reset on: COLOMBIA brought charges against b...,LATEST EDITORIAL
12,743842535,20180402,201804,2018,COLOMBIA,COL,NaN,NaN,190,8,...,-4.643963,CO,CO19,2.88915,-72.1256,6197118,https://www.telesurtv.net/english/news/One-Mor...,Villavicencio,teleSURHD Opacity If there is not a Board of ...,Guatemala: Tensions and Delay in the Formation...
13,744171630,20180403,201804,2018,RIGHTS GROUP,NaN,NaN,NaN,190,10,...,-4.713805,CO,CO19,2.88915,-72.1256,6197118,https://www.aljazeera.com/news/2018/03/rights-...,Villavicencio,"The Colombian social leader, known as an ‘unti...",Skip links
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16018,954967921,20201118,202011,2020,COLOMBIA,COL,NaN,NaN,190,12,...,-5.888438,CO,CO21,8.80389,-72.9536,29198,https://www.pressherald.com/2020/11/17/hurrica...,Cúcuta,\n\n\n\nSign in or Subscribe \nSee Offers\n\n\...,Sign In:
16019,954967922,20201118,202011,2020,COLOMBIAN,COL,NaN,NaN,190,25,...,-5.701540,CO,CO21,8.80389,-72.9536,29198,https://www.pressherald.com/2020/11/17/hurrica...,Cúcuta,\n\n\n\nSign in or Subscribe \nSee Offers\n\n\...,Sign In:
16021,955046947,20201118,202011,2020,PRESIDENT,NaN,NaN,NaN,190,3,...,-4.580153,CO,CO21,8.80389,-72.9536,29198,https://colombiareports.com/colombia-rescues-d...,Cúcuta,Colombia’s air force rescued more than 50 peop...,Colombia rescues dozens from island destroyed ...
16023,962369859,20210102,202101,2021,CARRIER,NaN,NaN,NaN,190,5,...,-1.003344,CO,CO21,8.27556,-72.3958,230744,https://www.flightglobal.com/safety/avianca-a3...,Cúcuta,NaN,NaN


In [8]:
# Filter Columns:
columns = ['Municipality code', 'SQLDATE', 'Actor1Name', 'Actor2Name','NumMentions', 'NumSources', 'NumArticles', 'AvgTone', 'SOURCEURL', 'ExtractedText', 'ContentTitle']

filtered_df = filtered_df[columns]

In [9]:
filtered_df.to_csv('labels_violence_cohort.csv', index=False)